In [489]:
import sleepy
import pandas as pd
import numpy as np
import math
import itertools
import plotly.express as px
from tqdm import tqdm
import time
import random
import multiprocessing as mp
from multiprocessing_functions import simulate_schedules, get_ranking

In [490]:
USERNAME = 'TheRealFergus'
YEAR = 2023

In [491]:
# Gets league and owner data
user_id = sleepy.get_user_data(USERNAME)["user_id"]
league_id = sleepy.get_league_ids(user_id, YEAR)[0]
league_raw = sleepy.get_leage(league_id)
owners_raw = sleepy.get_owners(league_id)
owners = owners_raw[["username", "owner_id", "roster_id"]]

In [492]:
# Stores the number of teams in the league
num_teams = owners.shape[0]

# Stores the number of the first week of the league playoffs
playoff_week1 = league_raw["settings"]["playoff_week_start"]

# Maps a unique matchup id corresponding to each combination of two roster ids
matchup_to_roster_id = {matchup[0] + 1: matchup[1] for matchup in 
                        enumerate(itertools.combinations(range(1,13), 2))}

roster_to_matchup_id = {val: key for (key, val) in matchup_to_roster_id.items()}

In [488]:
# Gets regular season matchup data
matchups = (sleepy.get_matchups(league_id, season=True)
            .query(f"starter == True & week < {playoff_week1}")
            .groupby(["week", "roster_id", "matchup_id"])
            [["team_points"]]
            .first()
            .reset_index())

# Merges matchups with owners to include usersernames
matchups = (
    matchups.merge(owners.reset_index(),
                    left_on="roster_id",
                      right_on = "roster_id")
            .assign(matchup_id = matchups["matchup_id"].astype(int))
            .drop(columns="index"))

In [6]:
# Gets regular season matchup data
matchups = (sleepy.get_matchups(league_id, season=True)
            .query(f"starter == True & week < {playoff_week1}")
            .groupby(["week", "roster_id", "matchup_id"])
            [["team_points"]]
            .first()
            .reset_index())

# Merges matchups with owners to include usersernames
matchups = (
    matchups.merge(owners.reset_index(),
                    left_on="roster_id",
                      right_on = "roster_id")
            .assign(matchup_id = matchups["matchup_id"].astype(int))
            .drop(columns="index"))

# Reassigns the matchup id
matchups["matchup_id"] = (
    matchups
    .groupby(["week", "matchup_id"])
    ["roster_id"]
    .transform(lambda x: roster_to_matchup_id[tuple(x.unique())])
    )

# The first week's games
matchups.head(12)

,week,roster_id,matchup_id,team_points,username,owner_id
0,1,1,10,95.92,alecwilson,781258862778015744
1,1,2,15,102.31,namebrant,737201118836346880
2,1,3,27,87.10,therealfergus,871830995287085056
3,1,4,36,80.88,empireyikesback,340376049508429824
4,1,5,40,75.52,pacc,791907251894984704
5,1,6,15,104.94,tonygordzilla22,790423754491678720
6,1,7,40,116.85,mackjyers21,463115290251620352
7,1,8,60,105.44,burgertownthicnred,865421962913157120
8,1,9,27,138.10,thezirconisdragon,865438032692649984
9,1,10,36,87.78,black8yellownation,865844843182694400


In [7]:
# Formats the actual season schedule
season_schedule = tuple(
    tuple(week) for week in matchups.groupby("week")["matchup_id"].agg(set)
)

# Gets the total season points for each owner
owners_points = matchups.groupby("username")[["team_points"]].sum()

In [8]:
# Gets all possible weekly schedules, each tuple contains six matchp id's 
# corresponding to a single game. There are 10395 possible weekly schedules
all_weeks = []
for week in tqdm(itertools.combinations(matchup_to_roster_id.values(), 6), 
                 total = math.comb(66, 6)):
    s = set()
    for match in week:
        s.update(match)
    if len(s) == 12:
        all_weeks.append(tuple(roster_to_matchup_id[match] for match in week))
all_weeks = tuple(all_weeks)

100%|██████████| 90858768/90858768 [00:45<00:00, 1996476.84it/s]


In [9]:
# Maps each week schedule in all possible weeks to a set of weeks. Given that the
# key week appears in a season schedule, none of the weeks in the value set can 
# also appear
similar_weeks = {}
for key_week in tqdm(all_weeks, total = len(all_weeks)):
    similar_weeks[key_week] = (
        set([week for week in all_weeks if len(set(week + key_week)) != 12])
        )

100%|██████████| 10395/10395 [00:24<00:00, 431.72it/s]


In [15]:
# Creates the shared data stuctures to hold results from different processors
manager = mp.Manager()
total_records = manager.dict()
ranking_counts = manager.dict()
for username in owners["username"]:
    total_records[username] = manager.list(np.zeros(12, dtype=int))

# Simulates schedules using multiple processors
if __name__ == "__main__":

    # Sets the number of simulations
    num_sims_per_process = 10000  # Number of simulations per process
    num_processes = 1000  # Number of processes (cores) to use
    total_sims = num_sims_per_process * num_processes

    processes = []

    for p in range(num_processes):

        # Continues while at the maximum number of active processes
        while len(processes) >= mp.cpu_count():
            time.sleep(1)
            processes = [p for p in processes if p.is_alive()]

        # When there is less than the max number of processess, start a new one
        process = mp.Process(
            target = simulate_schedules,
            args = (all_weeks, similar_weeks, owners, matchup_to_roster_id, matchups, owners_points, total_records, ranking_counts, num_sims_per_process)
        )
        process.start()
        processes.append(process)

    # Join remaining processes
    for process in processes:
        process.join()

# Formats and stores the results in CSVs
tot_rec = {username: pd.Series(list(lst), index = range(1,13)) 
           for username, lst in dict(total_records).items()}

records_df = (pd.DataFrame(tot_rec)
              .transpose()
              .sort_values(by = list(range(1,13)), ascending = False))

playoffs_df = (pd.DataFrame(dict(ranking_counts), index = ["Count"])
               .transpose()
               .reset_index()
               .query("Count > 0")
               .rename(columns={'level_0': 1, 'level_1': 2, 'level_2': 3,
                                'level_3': 4, 'level_4': 5, 'level_5': 6,
                                'level_6': 7, 'level_7': 8, 'level_8': 9,
                                'level_9': 10, 'level_10': 11, 'level_11': 12})
               .sort_values(by = 'Count', ascending = False))

records_df.to_csv(f'{YEAR}_{total_sims}_simulated_records.csv',
                   index_label = 'username')

playoffs_df.to_csv(f'{YEAR}_{total_sims}_simulated_playoffs.csv',
                    index = False)

In [394]:
# Reads previously stored simulated records
records_df = pd.read_csv(f'{YEAR}_{total_sims}_simulated_records.csv', 
                         index_col="username")
playoffs_df = pd.read_csv(f'{YEAR}_{total_sims}_simulated_playoffs.csv', 
                          index_col = False)

records_df_prop = records_df / (total_sims / 100)
records_df_count = records_df

In [395]:
# Stores the true season ranks
season_ranks = get_ranking(season_schedule, owners, matchup_to_roster_id, matchups, owners_points)
pd.DataFrame(season_ranks)

,rank
pacc,1
thezirconisdragon,2
herbietime,3
alecwilson,4
empireyikesback,5
burgertownthicnred,6
therealfergus,7
mackjyers21,8
shakylegs,9
tonygordzilla22,10


In [396]:
records_df_count

,1,2,3,4,5,6,7,8,9,10,11,12
username,,,,,,,,,,,,
herbietime,5191110,2466258,1263125,638384,281700,108655,38014,10242,2192,300,20,0
thezirconisdragon,2074697,2171155,2116925,1910464,908055,432684,211870,104144,44943,18354,6032,677
pacc,1301049,2361512,2587268,2017290,952164,439508,200778,89029,34619,12850,3618,315
alecwilson,1209358,2309450,2423388,2001998,1123941,556264,243843,95928,28520,6519,771,20
empireyikesback,100340,300327,652351,1275076,2215778,2300439,1551716,906495,443471,187408,59043,7556
therealfergus,79027,231050,524234,1112273,2232044,2101218,1603096,1077066,611100,301181,112416,15295
mackjyers21,29770,97803,239929,529663,1024433,1574398,2006172,2136204,1266054,688986,334023,72565
burgertownthicnred,12441,50768,148201,374902,863380,1594773,2346878,2212003,1351781,705852,291124,47897
shakylegs,1832,8321,27895,79198,204288,428686,818694,1486333,2669906,2335301,1514670,424876


In [397]:
# Highlights each user's actual ranking from the season in the simulated dataframe
def highlight_cells(row):
    col_to_highlight = season_ranks[row.name]
    return ['color: red' if int(col) == int(col_to_highlight) 
                            else '' for col in row.index]

highlighted_df = records_df_prop.astype(str).style.apply(highlight_cells,
                                                          axis = 1)
highlighted_df

,1,2,3,4,5,6,7,8,9,10,11,12
username,,,,,,,,,,,,
herbietime,51.9111,24.66258,12.63125,6.38384,2.817,1.08655,0.38014,0.10242,0.02192,0.003,0.0002,0.0
thezirconisdragon,20.74697,21.71155,21.16925,19.10464,9.08055,4.32684,2.1187,1.04144,0.44943,0.18354,0.06032,0.00677
pacc,13.01049,23.61512,25.87268,20.1729,9.52164,4.39508,2.00778,0.89029,0.34619,0.1285,0.03618,0.00315
alecwilson,12.09358,23.0945,24.23388,20.01998,11.23941,5.56264,2.43843,0.95928,0.2852,0.06519,0.00771,0.0002
empireyikesback,1.0034,3.00327,6.52351,12.75076,22.15778,23.00439,15.51716,9.06495,4.43471,1.87408,0.59043,0.07556
therealfergus,0.79027,2.3105,5.24234,11.12273,22.32044,21.01218,16.03096,10.77066,6.111,3.01181,1.12416,0.15295
mackjyers21,0.2977,0.97803,2.39929,5.29663,10.24433,15.74398,20.06172,21.36204,12.66054,6.88986,3.34023,0.72565
burgertownthicnred,0.12441,0.50768,1.48201,3.74902,8.6338,15.94773,23.46878,22.12003,13.51781,7.05852,2.91124,0.47897
shakylegs,0.01832,0.08321,0.27895,0.79198,2.04288,4.28686,8.18694,14.86333,26.69906,23.35301,15.1467,4.24876


In [398]:
def get_prob(username, rank, type = "equal"):
    """Gets the probability of a user ranking equal to, worse, or better than 
    they actually did. Probabilites are taken from the simulated rankings.

    Args:
        username (str): the username to get the probability ranking
        rank (int): the rank to determinge
        type (str, optional): Can be "equal", "worse", or "better". For each 
        option. Defaults to "equal".

    Returns:
        str: The function returns the probability of the given user ranking 
        equal/better/or worse than the passed rank. Probabilities are taken from
        the simulated records dataframe.
    """
    user_ix = records_df_prop.index.get_loc(username)
    if type == "equal":
        return records_df_prop.iloc[user_ix, rank-1]
    elif type == "worse":
        return records_df_prop.iloc[user_ix, rank:].sum()
    elif type == "better":
        return records_df_prop.iloc[user_ix, :rank-1].sum()
    elif type == "playoff":
        return records_df_prop.iloc[user_ix, :6].sum()

# Gets the probability dictionaries of worse/equal/better for each user
probs_worse = {username: get_prob(username, rank, type = "worse") for (username, rank) in season_ranks.items()}
probs_better = {username: get_prob(username, rank, type = "better") for (username, rank) in season_ranks.items()}
probs_equal = {username: get_prob(username, rank, type = "equal") for (username, rank) in season_ranks.items()}
probs_playoff = {username: get_prob(username, rank, type = "playoff") for (username, rank) in season_ranks.items()}

# Stores the probabilites of each user's ranking for the season
season_probs = (pd.DataFrame(season_ranks)
                .assign(Better = pd.Series(probs_better))
                .assign(Equal = pd.Series(probs_equal))
                .assign(Worse = pd.Series(probs_worse))
                .assign(Playoff = pd.Series(probs_playoff))
                .reset_index(names="username")
                .set_index("rank")
                .round(2))
season_probs

,username,Better,Equal,Worse,Playoff
rank,,,,,
1,pacc,0.00,13.01,86.99,96.59
2,thezirconisdragon,20.75,21.71,57.54,96.14
3,herbietime,76.57,12.63,10.80,99.49
4,alecwilson,59.42,20.02,20.56,96.24
5,empireyikesback,23.28,22.16,54.56,68.44
6,burgertownthicnred,14.50,15.95,69.56,30.44
7,therealfergus,62.80,16.03,21.17,62.80
8,mackjyers21,55.02,21.36,23.62,34.96
9,shakylegs,30.55,26.70,42.75,7.50


In [399]:
# Plots the season probabilities as a stacked bar chart
season_probs_melt = pd.melt(season_probs, id_vars=['username', "Playoff"], var_name=' ', value_name='Chance %')
fig = px.bar(
        season_probs_melt, 
        x='username', 
        y='Chance %', 
        color = ' ', 
        title = f'Chance of Ranking for 2023 Season',
        color_discrete_sequence = px.colors.qualitative.D3[:3][::-1],
        )

fig.update_layout(width=800,
                  height=400,
                  xaxis_title = None,
                  yaxis_title = "Chance %",
                  title_x = 0.5,
                  font=dict(size=12),
                  margin=dict(t=70, b=75, l=75, r=50),
                  bargap=0.5,
                  legend = dict(title = None,
                                font = dict(size=14))
                 )

In [400]:
# Plots a horizontal bar chart
season_probs_melt = pd.melt(season_probs.reset_index().sort_values(by="rank", ascending=False).set_index("rank"), id_vars=['username', "Playoff"], var_name=' ', value_name='Chance %')

# Create the horizontal bar chart
fig = px.bar(
    season_probs_melt, 
    y='username', 
    x='Chance %', 
    color=' ', 
    title=f'Chance of Ranking for 2023 Season',
    orientation='h',  # Horizontal bar chart
    color_discrete_sequence=px.colors.qualitative.D3[:3][::-1],
)

# Update the layout
fig.update_layout(
    width=800,
    height=800,
    xaxis_title="Chance %",
    yaxis_title=None,
    title_x=0.5,
    font=dict(size=12),
    margin=dict(t=70, b=75, l=75, r=50),
    bargap=0.5,
    legend=dict(
        title=None,
        font=dict(size=16)
    )
)

fig.show()

In [401]:
fig.write_html('Visualizations/2023Probabilities.html', include_plotlyjs='cdn')

In [402]:
playoffs_df

,1,2,3,4,5,6,7,8,9,10,11,12,Count
0,herbietime,alecwilson,pacc,thezirconisdragon,therealfergus,empireyikesback,burgertownthicnred,mackjyers21,shakylegs,namebrant,tonygordzilla22,black8yellownation,15035
1,herbietime,thezirconisdragon,alecwilson,pacc,therealfergus,empireyikesback,burgertownthicnred,mackjyers21,shakylegs,namebrant,tonygordzilla22,black8yellownation,9599
2,thezirconisdragon,herbietime,alecwilson,pacc,therealfergus,empireyikesback,burgertownthicnred,mackjyers21,shakylegs,namebrant,tonygordzilla22,black8yellownation,9549
3,herbietime,pacc,thezirconisdragon,alecwilson,therealfergus,empireyikesback,burgertownthicnred,mackjyers21,shakylegs,namebrant,tonygordzilla22,black8yellownation,8782
4,herbietime,alecwilson,pacc,thezirconisdragon,empireyikesback,therealfergus,burgertownthicnred,mackjyers21,shakylegs,namebrant,tonygordzilla22,black8yellownation,7777
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042661,herbietime,therealfergus,pacc,alecwilson,thezirconisdragon,mackjyers21,burgertownthicnred,tonygordzilla22,empireyikesback,namebrant,shakylegs,black8yellownation,1
1042662,herbietime,thezirconisdragon,therealfergus,empireyikesback,pacc,mackjyers21,burgertownthicnred,shakylegs,alecwilson,namebrant,tonygordzilla22,black8yellownation,1
1042663,pacc,herbietime,alecwilson,empireyikesback,thezirconisdragon,namebrant,therealfergus,shakylegs,mackjyers21,burgertownthicnred,tonygordzilla22,black8yellownation,1
1042664,thezirconisdragon,alecwilson,empireyikesback,herbietime,pacc,namebrant,shakylegs,therealfergus,burgertownthicnred,mackjyers21,black8yellownation,tonygordzilla22,1


In [403]:
# ge_df_prob contains the chance of each team ranking reater than or equal to 
# the rank achieved
ge_df_prob = pd.DataFrame(playoffs_df)

def better_equal(username, rank):
    return get_prob(username, rank, type = "better") + get_prob(username, rank, type = "equal")

for i in tqdm(range(1,13), total = 12):
    ge_df_prob[str(i)] = playoffs_df[str(i)].apply(lambda x: better_equal(x,i))

100%|██████████| 12/12 [05:54<00:00, 29.53s/it]


In [404]:
# The ranking that occured most often, 15035 times
idx_most_likely_ranking = 1

most_likely_ranking = pd.concat(
    [playoffs_df.iloc[idx_most_likely_ranking,:12].rename("Username"),
     ge_df_prob.iloc[idx_most_likely_ranking,:12].rename("Chance >=")],
     axis=1).reset_index(names="Rank").merge(
         season_probs[["username", "Playoff"]], 
         left_on = "Username",
         right_on = "username"
     ).drop(columns="username")

most_likely_ranking

,Rank,Username,Chance >=,Playoff
0,1,herbietime,51.91110,99.49
1,2,thezirconisdragon,42.45852,96.14
2,3,alecwilson,59.42196,96.24
3,4,pacc,82.67119,96.59
4,5,therealfergus,41.78628,62.80
5,6,empireyikesback,68.44311,68.44
6,7,burgertownthicnred,53.91343,30.44
7,8,mackjyers21,76.38372,34.96
8,9,shakylegs,57.25153,7.50
9,10,namebrant,68.99480,5.02


In [481]:
# The ranking with the smallest combined chance of greater than or equal to in
# the first three rankings
idx_unlikely = ge_df_prob.iloc[:,:3].sum(axis = 1).idxmin()
unlikely_top_3 = (pd.concat(
    [playoffs_df.iloc[idx_unlikely,:12].rename("Username"),
     ge_df_prob.iloc[idx_unlikely,:12].rename("Chance >=")],
     axis = 1)).reset_index(names="Rank").merge(
         season_probs[["username", "Playoff"]], 
         left_on = "Username",
         right_on = "username"
     ).drop(columns="username")
unlikely_top_3

,Rank,Username,Chance >=,Playoff
0,1,mackjyers21,0.29770,34.96
1,2,burgertownthicnred,0.63209,30.44
2,3,shakylegs,0.38048,7.50
3,4,herbietime,95.58877,99.49
4,5,alecwilson,90.68135,96.24
5,6,pacc,96.58791,96.59
6,7,thezirconisdragon,98.25850,96.14
7,8,therealfergus,89.60008,62.80
8,9,empireyikesback,97.45993,68.44
9,10,namebrant,68.99480,5.02


In [408]:
# playoff_df_prob contains the chance of each team making the playoffs
playoff_df_prob = pd.DataFrame(playoffs_df)

def better_equal(username, rank):
    return get_prob(username, rank, type = "playoff")

for i in tqdm(range(1,13), total = 12):
    playoff_df_prob[str(i)] = playoffs_df[str(i)].apply(lambda x: better_equal(x,i))

100%|██████████| 12/12 [04:37<00:00, 23.09s/it]


In [484]:
# The ranking with the lowest combined playoff percentage
idx_unlikely_playoff = playoff_df_prob.iloc[:, :6].product(axis=1).idxmin()
unlikely_playoff_seed2 = (pd.concat(
    [playoffs_df.iloc[idx_unlikely_playoff,:12].rename("Username"),
     ge_df_prob.iloc[idx_unlikely_playoff,:12].rename("Chance >=")],
     axis = 1)).reset_index(names="Rank").merge(
         season_probs[["username", "Playoff"]], 
         left_on = "Username",
         right_on = "username"
     ).drop(columns="username")
unlikely_playoff_seed2

,Rank,Username,Chance >=,Playoff
0,1,herbietime,51.91110,99.49
1,2,pacc,36.62561,96.59
2,3,thezirconisdragon,63.62777,96.14
3,4,namebrant,0.63411,5.02
4,5,tonygordzilla22,0.72256,2.24
5,6,black8yellownation,0.12402,0.12
6,7,alecwilson,98.68242,96.24
7,8,empireyikesback,93.02522,68.44
8,9,burgertownthicnred,89.55127,30.44
9,10,therealfergus,98.72289,62.80


In [244]:
# This cell estimates the total number possible season schedules.

def generate_schedule_nums():
    """This funciton is a modified verstion of the generate schedule function 
    used in the multiprocessing function file. It aims to estimate the total 
    number of possible schedules. 
    
    Generates a 14 week season schedule for an 12 team leage. Each team plays
    all other teams once in the first 11 weeks. The first 3 weeks are repeated
    for the last 3 weeks.

    Returns:
        list: A list of the number of schedules to choose from at each week
    each week.
    """
    weeks = set(all_weeks)
    schedule = []
    num_options = []

    # Recursively fills the weekly schedule at random
    def pick_week(weeks):
        if len(schedule) == 11:
            return
        else:
            tuple_weeks = tuple(weeks)
            num_options.append(len(tuple_weeks))
            choice = random.choice(tuple_weeks)
            schedule.append(choice)
            weeks -= similar_weeks[choice]
            pick_week(weeks)
    
    # The above method has about a 70% success rate to pick a valid yearly 
    # schedule. This while loop will continue until a valid schedule is picked.
    while len(schedule) != 11:
        try:
            pick_week(weeks)
        except IndexError:
            weeks = set(all_weeks)
            schedule = []
            num_options = []

    # Adds the first three weeks of the schedule to the end to finish the 14
    # week season
    schedule += schedule[:3]

    total = 1
    for i in num_options:
        total *= i

    return(total)

np.array([generate_schedule_nums() for i in range(1000)]).mean() * .7

1.011275136797421e+25

In [319]:
# Prints the season outcomes in html
outcome_2023 = owners_raw[["username",
                           "fpts",
                           "fpts_decimal", 
                           "fpts_against", 
                           "fpts_against_decimal", 
                           "wins", 
                           "losses"]]

def format_dec(x):
    out = str((x/100))[1:]
    if len(out) == 2:
        out = out + "0"
    return out

outcome_2023 = (
    outcome_2023
    .assign(points_for = outcome_2023["fpts"].astype(str) 
            + (outcome_2023["fpts_decimal"].apply(format_dec)))
    .assign(points_against = outcome_2023["fpts_against"].astype(str) 
            + (outcome_2023["fpts_against_decimal"].apply(format_dec)))
    .assign(record = outcome_2023["wins"].astype(str) + "-" 
            + outcome_2023["losses"].astype(str))
            )
outcome_2023 = pd.DataFrame(season_ranks).merge(
        outcome_2023,
        left_index=True, 
        right_on='username')


outcome_2023 = (outcome_2023
                .set_index("rank")
                .drop(columns=["fpts", "fpts_decimal", "fpts_against", 
                               "fpts_against_decimal", "wins", "losses"])
                .reset_index()
                .rename(columns={"rank":"Rank",
                                 "username": "Username",
                                 "points_for":"Points For",
                                 "points_against":"Points Against",
                                 "record":"Record"})
)
print(outcome_2023.to_html(index=False))

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Rank</th>
      <th>Username</th>
      <th>Points For</th>
      <th>Points Against</th>
      <th>Record</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>pacc</td>
      <td>2008.96</td>
      <td>1765.22</td>
      <td>9-5</td>
    </tr>
    <tr>
      <td>2</td>
      <td>thezirconisdragon</td>
      <td>2000.73</td>
      <td>1895.99</td>
      <td>9-5</td>
    </tr>
    <tr>
      <td>3</td>
      <td>herbietime</td>
      <td>2114.47</td>
      <td>1990.59</td>
      <td>8-6</td>
    </tr>
    <tr>
      <td>4</td>
      <td>alecwilson</td>
      <td>2107.20</td>
      <td>1854.36</td>
      <td>8-6</td>
    </tr>
    <tr>
      <td>5</td>
      <td>empireyikesback</td>
      <td>1798.42</td>
      <td>1662.93</td>
      <td>8-6</td>
    </tr>
    <tr>
      <td>6</td>
      <td>burgertownthicnred</td>
      <td>1769.67</td>
      <td>1778.89</td>
      <td>8-6</td

In [312]:
# Prints the records count in html
records_df_count.index.name=None
print(records_df_count.astype(str).to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>1</th>
      <th>2</th>
      <th>3</th>
      <th>4</th>
      <th>5</th>
      <th>6</th>
      <th>7</th>
      <th>8</th>
      <th>9</th>
      <th>10</th>
      <th>11</th>
      <th>12</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>herbietime</th>
      <td>5191110</td>
      <td>2466258</td>
      <td>1263125</td>
      <td>638384</td>
      <td>281700</td>
      <td>108655</td>
      <td>38014</td>
      <td>10242</td>
      <td>2192</td>
      <td>300</td>
      <td>20</td>
      <td>0</td>
    </tr>
    <tr>
      <th>thezirconisdragon</th>
      <td>2074697</td>
      <td>2171155</td>
      <td>2116925</td>
      <td>1910464</td>
      <td>908055</td>
      <td>432684</td>
      <td>211870</td>
      <td>104144</td>
      <td>44943</td>
      <td>18354</td>
      <td>6032</td>
      <td>677</td>
    </tr>
    <tr>
      <th>pacc</th>
      <td>1301049</td>
  

In [322]:
# Prints the highlighted percentage in html
highlighted_df.index.name=None
print(highlighted_df.to_html())

<style type="text/css">
#T_8d0e4_row0_col2, #T_8d0e4_row1_col1, #T_8d0e4_row2_col0, #T_8d0e4_row3_col3, #T_8d0e4_row4_col4, #T_8d0e4_row5_col6, #T_8d0e4_row6_col7, #T_8d0e4_row7_col5, #T_8d0e4_row8_col8, #T_8d0e4_row9_col11, #T_8d0e4_row10_col9, #T_8d0e4_row11_col10 {
  color: red;
}
</style>
<table id="T_8d0e4">
  <thead>
    <tr>
      <th class="blank level0" >&nbsp;</th>
      <th id="T_8d0e4_level0_col0" class="col_heading level0 col0" >1</th>
      <th id="T_8d0e4_level0_col1" class="col_heading level0 col1" >2</th>
      <th id="T_8d0e4_level0_col2" class="col_heading level0 col2" >3</th>
      <th id="T_8d0e4_level0_col3" class="col_heading level0 col3" >4</th>
      <th id="T_8d0e4_level0_col4" class="col_heading level0 col4" >5</th>
      <th id="T_8d0e4_level0_col5" class="col_heading level0 col5" >6</th>
      <th id="T_8d0e4_level0_col6" class="col_heading level0 col6" >7</th>
      <th id="T_8d0e4_level0_col7" class="col_heading level0 col7" >8</th>
      <th id="T_8d0e4_

In [487]:
# Prints season_probs in html
print(season_probs.reset_index().rename(columns={"rank":"Rank",
                                                 "username":"Username"})
                                                 .to_html(index=False))

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Rank</th>
      <th>Username</th>
      <th>Better</th>
      <th>Equal</th>
      <th>Worse</th>
      <th>Playoff</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>pacc</td>
      <td>0.00</td>
      <td>13.01</td>
      <td>86.99</td>
      <td>96.59</td>
    </tr>
    <tr>
      <td>2</td>
      <td>thezirconisdragon</td>
      <td>20.75</td>
      <td>21.71</td>
      <td>57.54</td>
      <td>96.14</td>
    </tr>
    <tr>
      <td>3</td>
      <td>herbietime</td>
      <td>76.57</td>
      <td>12.63</td>
      <td>10.80</td>
      <td>99.49</td>
    </tr>
    <tr>
      <td>4</td>
      <td>alecwilson</td>
      <td>59.42</td>
      <td>20.02</td>
      <td>20.56</td>
      <td>96.24</td>
    </tr>
    <tr>
      <td>5</td>
      <td>empireyikesback</td>
      <td>23.28</td>
      <td>22.16</td>
      <td>54.56</td>
      <td>68.44</td>
    </tr>
    <tr>
      <td>6

In [460]:
# Prints most_likely_ranking in html
print(most_likely_ranking.to_html(index=False))

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Rank</th>
      <th>Username</th>
      <th>Chance &gt;=</th>
      <th>Playoff</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>herbietime</td>
      <td>51.91110</td>
      <td>99.49</td>
    </tr>
    <tr>
      <td>2</td>
      <td>thezirconisdragon</td>
      <td>42.45852</td>
      <td>96.14</td>
    </tr>
    <tr>
      <td>3</td>
      <td>alecwilson</td>
      <td>59.42196</td>
      <td>96.24</td>
    </tr>
    <tr>
      <td>4</td>
      <td>pacc</td>
      <td>82.67119</td>
      <td>96.59</td>
    </tr>
    <tr>
      <td>5</td>
      <td>therealfergus</td>
      <td>41.78628</td>
      <td>62.80</td>
    </tr>
    <tr>
      <td>6</td>
      <td>empireyikesback</td>
      <td>68.44311</td>
      <td>68.44</td>
    </tr>
    <tr>
      <td>7</td>
      <td>burgertownthicnred</td>
      <td>53.91343</td>
      <td>30.44</td>
    </tr>
    <tr>
      <td>8</td>

In [482]:
# Prints unlikely_top_3 in html
print(unlikely_top_3.to_html(index=False))

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Rank</th>
      <th>Username</th>
      <th>Chance &gt;=</th>
      <th>Playoff</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>mackjyers21</td>
      <td>0.29770</td>
      <td>34.96</td>
    </tr>
    <tr>
      <td>2</td>
      <td>burgertownthicnred</td>
      <td>0.63209</td>
      <td>30.44</td>
    </tr>
    <tr>
      <td>3</td>
      <td>shakylegs</td>
      <td>0.38048</td>
      <td>7.50</td>
    </tr>
    <tr>
      <td>4</td>
      <td>herbietime</td>
      <td>95.58877</td>
      <td>99.49</td>
    </tr>
    <tr>
      <td>5</td>
      <td>alecwilson</td>
      <td>90.68135</td>
      <td>96.24</td>
    </tr>
    <tr>
      <td>6</td>
      <td>pacc</td>
      <td>96.58791</td>
      <td>96.59</td>
    </tr>
    <tr>
      <td>7</td>
      <td>thezirconisdragon</td>
      <td>98.25850</td>
      <td>96.14</td>
    </tr>
    <tr>
      <td>8</td>
      <td>t

In [485]:
# Prints unlikely_playoff_seed2 in html
print(unlikely_playoff_seed2.to_html(index=False))

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Rank</th>
      <th>Username</th>
      <th>Chance &gt;=</th>
      <th>Playoff</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>herbietime</td>
      <td>51.91110</td>
      <td>99.49</td>
    </tr>
    <tr>
      <td>2</td>
      <td>pacc</td>
      <td>36.62561</td>
      <td>96.59</td>
    </tr>
    <tr>
      <td>3</td>
      <td>thezirconisdragon</td>
      <td>63.62777</td>
      <td>96.14</td>
    </tr>
    <tr>
      <td>4</td>
      <td>namebrant</td>
      <td>0.63411</td>
      <td>5.02</td>
    </tr>
    <tr>
      <td>5</td>
      <td>tonygordzilla22</td>
      <td>0.72256</td>
      <td>2.24</td>
    </tr>
    <tr>
      <td>6</td>
      <td>black8yellownation</td>
      <td>0.12402</td>
      <td>0.12</td>
    </tr>
    <tr>
      <td>7</td>
      <td>alecwilson</td>
      <td>98.68242</td>
      <td>96.24</td>
    </tr>
    <tr>
      <td>8</td>
      <td